In [1]:
import torch
import numpy as np
import gymnasium as gym
import mujoco
from Diffusion import PoseDiffusion, QuadrupedMorphDiffusionDataset


def load_diffusion(
    weights_path="quadruped_morph_diffusion_weights.pt",
    traj_file="quadruped_morph_trajectories.npz",
    device=None ):

    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Using device:", device)

    # Load checkpoint and config
    ckpt = torch.load(weights_path, map_location=device)
    cfg = ckpt["config"]

    past_len   = cfg["past_len"]
    future_len = cfg["future_len"]
    n_steps    = cfg["n_steps"]
    hidden     = cfg["hidden"]
    imitation_obs_indices = cfg.get("imitation_obs_indices", None)

    print("Loaded config from checkpoint:")
    print("  traj_file :", traj_file)
    print("  past_len  :", past_len)
    print("  future_len:", future_len)
    print("  n_steps   :", n_steps)
    print("  hidden    :", hidden)
    print("  imitation_obs_indices:", imitation_obs_indices)


    # Rebuild dataset to get obs_dim / cond_dim
    dataset = QuadrupedMorphDiffusionDataset(
        traj_path=traj_file,
        past_len=past_len,
        future_len=future_len,
        imitation_obs_indices=imitation_obs_indices,
        device=device
    )

    D_target = dataset.D_target   # obs_dim per time step
    D_cond   = dataset.D_cond     # past_len*obs_dim + morph_dim

    print("Reconstructed diffusion dims:")
    print("  D_target (obs_dim) =", D_target)
    print("  D_cond             =", D_cond)

    diff_model = PoseDiffusion(
        D_target=D_target,
        D_cond=D_cond,
        future_len=future_len,
        n_steps=n_steps,
        hidden=hidden,
    ).to(device)

    diff_model.load_state_dict(ckpt["state_dict"])
    diff_model.eval()

    print("Loaded diffusion model.")

    # Return metadata for sampling
    meta = {
        "past_len": past_len,
        "future_len": future_len,
        "obs_dim": D_target,
        "morph_dim": dataset.morph_dim,
        "D_target": D_target,
        "D_cond": D_cond,
        "imitation_obs_indices": imitation_obs_indices,
    }

    return diff_model, meta, device

def make_cond_from_past_and_morph(past_obs, morph_vec, ckpt, device):
    """
    past_obs: (past_len, obs_dim)
    morph_vec: (morph_dim,)
    """
    past_len = ckpt["past_len"]
    obs_dim  = ckpt["obs_dim"]

    past_obs = np.asarray(past_obs, dtype=np.float32)
    morph_vec = np.asarray(morph_vec, dtype=np.float32)

    assert past_obs.shape == (past_len, obs_dim), (
        f"past_obs shape {past_obs.shape} != ({past_len}, {obs_dim})"
    )

    past_flat = past_obs.reshape(-1)                # (past_len * obs_dim,)
    cond_np   = np.concatenate([past_flat, morph_vec], axis=0)

    cond = torch.tensor(cond_np, device=device, dtype=torch.float32).unsqueeze(0)
    # shape: (1, D_cond)
    return cond

def sample_future_obs(diff_model, cond, meta):
    future_len = meta["future_len"]
    obs_dim    = meta["obs_dim"]

    with torch.no_grad():
        x0_seq = diff_model.sample(cond)   # (1, future_len, obs_dim)

    future_obs = x0_seq[0].cpu().numpy()   # (future_len, obs_dim)
    return future_obs

def pred_to_qpos_from_diffusion(pred, env):
    pred = np.asarray(pred, dtype=np.float32)
    qpos = env.unwrapped.data.qpos.copy()

    nq = env.unwrapped.model.nq  # 15 for Ant
    root_dim = 7 #root position not needed
    max_joints = nq - root_dim    # number of slots after x,y

    L = min(pred.shape[0], max_joints)
    qpos[root_dim:root_dim+L] = pred[:L]

    return qpos

def obs_to_qpos_from_augmented(obs_aug, env):
    obs_aug = np.asarray(obs_aug, dtype=np.float32)
    root_dim = 7 #root position not needed
    morph_dim = 8
    
    nq = env.unwrapped.model.nq  # 15
    joint_dim = nq - root_dim
    
    base_obs_dim = obs_aug.shape[0] - (morph_dim + 2)  # 97 - 10 = 87
    base_obs = obs_aug[:base_obs_dim]
    qpos = env.unwrapped.data.qpos.copy()   

    qpos[root_dim:] = base_obs[:joint_dim]
    return qpos

# freeze frame simulated gait cycle
def make_diffusion_video(full_cycle, xml_path, save_dir="diff_video", fps=30):
    env = gym.make("Ant-v5", xml_file=xml_path, render_mode="rgb_array" )

    env = gym.wrappers.RecordVideo( env, video_folder=save_dir, 
                                    name_prefix="diffusion_gait",
                                    episode_trigger=lambda ep: True )

    obs, info = env.reset()
    model = env.unwrapped.model
    data = env.unwrapped.data
    T = full_cycle.shape[0]
    morph_dim=8
    frame_count = 0        

    for t in range(T):
        pred = full_cycle[t] 

        qpos = pred_to_qpos_from_diffusion(pred, env)
        data.qpos[:] = qpos
        data.qvel[:] = 0.0
        mujoco.mj_forward(model, data)

        # Take astep just to trigger render & RecordVideo
        zero_action = np.zeros(env.action_space.shape, dtype=np.float32)
        obs, reward, done, truncated, info = env.step(zero_action)
        frame_count += 1 

    env.close()
    print(f"Frames rendered: {frame_count}")
    print(f"Video saved to {save_dir}")

In [2]:
### Diff Video ###

import Diffusion as diff
from Diffusion import DiffusionPoseTemplate



quad_morph = [
    0.141421, 0.282843,  # FR leg, FR ankle
    0.141421, 0.282843,  # FL leg, FL ankle
    0.141421, 0.282843,  # BL leg, BL ankle
    0.141421, 0.282843,  # BR leg, BR ankle
]

diff_model, meta, device = load_diffusion(
    weights_path="quadruped_morph_diffusion_weights.pt",
    traj_file="quadruped_morph_trajectories.npz",)


full_cycle = diff.sample_full_cycle_from_morph(
    diff_model,
    meta,
    quad_morph,
    cycle_steps=200,
    device="cuda",
)

print(full_cycle.shape)
print(full_cycle[0])


#make_diffusion_video(full_cycle, "./quadruped.xml")


Using device: cuda
Loaded config from checkpoint:
  traj_file : quadruped_morph_trajectories.npz
  past_len  : 10
  future_len: 200
  n_steps   : 100
  hidden    : 256
  imitation_obs_indices: [5, 6, 7, 8, 9, 10, 11, 12]
[QuadrupedDataset] obs_dim=97, morph_dim=8
[QuadrupedDataset] target_dim=8
[QuadrupedDataset] total samples possible=87182
Reconstructed diffusion dims:
  D_target (obs_dim) = 8
  D_cond             = 88
Loaded diffusion model.
(200, 8)
[ 0.20212217  0.03245725  0.38002956 -0.5836979   0.16443215 -0.4536708
 -0.20561464  0.53085864]


In [3]:
#Train the Cycle with Imitation

import Reinforcement as re

#standard obs indices for Ant joints
imitation_obs_indices = list(range(5, 13))

# Morph of interest
quad_morph = [
    0.141421, 0.282843,
    0.141421, 0.282843,
    0.141421, 0.282843,
    0.141421, 0.282843,
]


pose_template = DiffusionPoseTemplate(full_cycle)

re.train_ppo_with_pose_template(
    run_name="ppo_with_imitation",
    pose_generator=pose_template,
    morph_vec=quad_morph,  
    xml_path="./quadruped.xml",
    timesteps=1_000_000,
    parallel_envs=4,
    initial_learning_rate=3e-4,
    imitation_w=1,
    imitation_obs_indices=imitation_obs_indices
)

Logging to ./logs_ppo_with_imitation
Using cuda device


C:\Users\Lep\miniconda3\Lib\site-packages\stable_baselines3\common\policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(
C:\Users\Lep\miniconda3\Lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(



🚀 Training PPO with pose templates for ppo_with_imitation ...
[env 1] ep_len=  51 | R_mean=-0.323 | fwd=-0.608 | vel=-1.982 | imitat=-0.008 | alive= 0.100 | energy_p= 0.005 | fail= 0.294 | phase_spd= 0.044 | var=unknown
[env 3] ep_len=  57 | R_mean=-0.239 | fwd=-1.230 | vel=-4.005 | imitat=-0.006 | alive= 0.100 | energy_p= 0.005 | fail= 0.263 | phase_spd= 0.042 | var=unknown
[env 0] ep_len=  61 | R_mean= 0.012 | fwd=-0.200 | vel=-0.684 | imitat=-0.001 | alive= 0.100 | energy_p= 0.004 | fail= 0.246 | phase_spd= 0.047 | var=unknown
[env 2] ep_len=  90 | R_mean=-0.136 | fwd=-0.660 | vel=-2.176 | imitat=-0.001 | alive= 0.100 | energy_p= 0.004 | fail= 0.167 | phase_spd= 0.050 | var=unknown
[env 1] ep_len=  51 | R_mean=-0.075 | fwd=-0.886 | vel=-2.860 | imitat= 0.000 | alive= 0.100 | energy_p= 0.004 | fail= 0.294 | phase_spd= 0.038 | var=unknown
[env 3] ep_len=  54 | R_mean=-0.028 | fwd=-0.284 | vel=-0.938 | imitat= 0.000 | alive= 0.100 | energy_p= 0.004 | fail= 0.278 | phase_spd= 0.048 | v

C:\Users\Lep\miniconda3\Lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at C:\Users\Lep\OneDrive\Documents\Course Work\CS 525 Brain In Comp\Project\genloco\quad_videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Video recording complete.
[env 1] ep_len=  55 | R_mean=-0.086 | fwd=-0.711 | vel=-2.306 | imitat=-0.013 | alive= 0.100 | energy_p= 0.006 | fail= 0.273 | phase_spd= 0.046 | var=unknown
[env 3] ep_len=  94 | R_mean=-0.023 | fwd=-0.157 | vel=-0.561 | imitat=-0.012 | alive= 0.100 | energy_p= 0.006 | fail= 0.160 | phase_spd= 0.047 | var=unknown
[env 1] ep_len=  74 | R_mean=-0.016 | fwd=-0.079 | vel=-0.317 | imitat=-0.012 | alive= 0.100 | energy_p= 0.006 | fail= 0.203 | phase_spd= 0.044 | var=unknown
[env 0] ep_len= 177 | R_mean=-0.009 | fwd=-0.052 | vel=-0.213 | imitat=-0.013 | alive= 0.100 | energy_p= 0.006 | fail= 0.085 | phase_spd= 0.043 | var=unknown
[env 3] ep_len= 129 | R_mean=-0.024 | fwd=-0.172 | vel=-0.635 | imitat=-0.013 | alive= 0.100 | energy_p= 0.006 | fail= 0.116 | phase_spd= 0.045 | var=unknown
[env 2] ep_len= 266 | R_mean=-0.012 | fwd=-0.082 | vel=-0.326 | imitat=-0.012 | alive= 0.100 | energy_p= 0.006 | fail= 0.056 | phase_spd= 0.046 | var=unknown
[env 1] ep_len=  96 | R_me

In [6]:
import numpy as np

data = np.load("quadruped_morph_trajectories.npz")
obs = data["obs"]
morph = data["morph"]

print("obs_dim:", obs.shape[1])
print("morph_dim:", morph.shape[1])

obs_dim: 97
morph_dim: 8


In [3]:
import Reinforcement as re
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv, VecNormalize, VecMonitor, VecVideoRecorder
from stable_baselines3.common.utils import LinearSchedule
from stable_baselines3.common.logger import configure

def evaluate_quadruped_ppo(
    ppo_path: str,
    xml_path: str,
    morph_vec,
    episodes: int = 10,
    max_steps_per_ep: int = 1000,
    deterministic: bool = True,
    pose_generator=None,
    imitation_w: float = 0.0,
    imitation_obs_indices=None ):

    print(f"Loading PPO model from {ppo_path} ...")
    model = PPO.load(ppo_path)

    # Base env: same quadruped XML as in training
    base_env = re.make_quadruped_env(seed=0, xml_path=xml_path)

    # Wrap in MorphPhaseEnvWrapper so we get delta_x and energy_penalty in info
    eval_env = re.MorphPhaseEnvWrapper(
        base_env=base_env,
        morph_vec=morph_vec,
        cycle_steps=200,
        settle_steps=5,
        pose_generator=pose_generator,
        imitation_w=imitation_w,
        imitation_obs_indices=imitation_obs_indices,
        xml_path=xml_path,
    )

    episode_forward = []
    episode_energy = []

    for ep in range(episodes):
        obs, info = eval_env.reset()
        done = False
        truncated = False
        ep_forward = 0.0
        ep_energy = 0.0

        for t in range(max_steps_per_ep):
            # stable-baselines3 expects obs shape (n_envs, obs_dim) for VecEnv,
            # but here we have a single Gym env, so squeeze/unsqueeze as needed:
            action, _ = model.predict(obs, deterministic=deterministic)
            obs, reward, done, truncated, info = eval_env.step(action)

            # info fields are populated by MorphPhaseEnvWrapper.step(...)
            ep_forward += float(info.get("delta_x", 0.0))
            ep_energy  += float(info.get("energy_penalty", 0.0))

            if done or truncated:
                break

        episode_forward.append(ep_forward)
        episode_energy.append(ep_energy)
        print(
            f"Episode {ep+1:2d} | "
            f"forward = {ep_forward:.3f} | "
            f"energy = {ep_energy:.3f} | "
            f"steps = {t+1}"
        )

    avg_forward = float(np.mean(episode_forward)) if episode_forward else 0.0
    avg_energy  = float(np.mean(episode_energy)) if episode_energy else 0.0

    print("\n=== PPO evaluation over "
          f"{episodes} episodes ({xml_path}) ===")
    print(f"Average forward motion: {avg_forward:.3f}")
    print(f"Average energy spent:   {avg_energy:.3f}")

    return avg_forward, avg_energy
    

In [4]:
#Metrics Forward movement and energy

quad_morph = [
    0.141421, 0.282843,
    0.141421, 0.282843,
    0.141421, 0.282843,
    0.141421, 0.282843,
]

var1_morph = [
    0.2, 0.25,  # FR leg, FR ankle
    0.2, 0.25,  # FL leg, FL ankle
    0.141421, 0.3,  # BL leg, BL ankle
    0.141421, 0.3,  # BR leg, BR ankle
]

explore_var = "quadrl_var1_ppo.zip"
mlp = "ppo_with_mlp.zip"
generative = "ppo_with_imitation_ppo.zip"

diff_model, meta, device = load_diffusion(
    weights_path="quadruped_morph_diffusion_weights.pt",
    traj_file="quadruped_morph_trajectories.npz",)

full_cycle = diff.sample_full_cycle_from_morph(
    diff_model,
    meta,
    quad_morph,
    cycle_steps=200,
    device="cuda",
)
pose_template = DiffusionPoseTemplate(full_cycle)


avg_fwd1, avg_energy1 = evaluate_quadruped_ppo(
    ppo_path=explore_var,   
    xml_path="./quadruped_var1.xml",
    morph_vec=quad_morph,
    episodes=20,
    max_steps_per_ep=1000,
    deterministic=True,
    pose_generator=None,   
    imitation_w=0.0,    
    imitation_obs_indices=None,
)

avg_fwd2, avg_energy2 = evaluate_quadruped_ppo(
    ppo_path=mlp,   
    xml_path="./quadruped.xml",
    morph_vec=quad_morph,
    episodes=20,
    max_steps_per_ep=1000,
    deterministic=True,
    pose_generator=None,   
    imitation_w=0.0,    
    imitation_obs_indices=None,
)

avg_fwd3, avg_energy3 = evaluate_quadruped_ppo(
    ppo_path=generative,   
    xml_path="./quadruped.xml",
    morph_vec=quad_morph,
    episodes=20,
    max_steps_per_ep=1000,
    deterministic=True,
    pose_generator=pose_template,    # DiffusionPoseTemplate active
    imitation_w=0.0,    
    imitation_obs_indices=None,
)


print(" Quadruped PPO Evaluation Summary")
print("")

header = f"{'Model':<20} | {'Avg Forward':>12} | {'Avg Energy':>12} | {'Notes'}"
print(header)
print("-"*72)

rows = [
    ("Exploration-PPO",  avg_fwd1, avg_energy1, "baseline (explore_var)"),
    ("MLP-PPO",          avg_fwd2, avg_energy2, "baseline (mlp)"),
    ("Generative-PPO",   avg_fwd3, avg_energy3, "diffusion-guided"),
]

for name, fwd, energy, note in rows:
    print(f"{name:<20} | {fwd:12.4f} | {energy:12.4f} | {note}")

Using device: cuda
Loaded config from checkpoint:
  traj_file : quadruped_morph_trajectories.npz
  past_len  : 10
  future_len: 200
  n_steps   : 100
  hidden    : 256
  imitation_obs_indices: [5, 6, 7, 8, 9, 10, 11, 12]
[QuadrupedDataset] obs_dim=97, morph_dim=8
[QuadrupedDataset] target_dim=8
[QuadrupedDataset] total samples possible=87182
Reconstructed diffusion dims:
  D_target (obs_dim) = 8
  D_cond             = 88
Loaded diffusion model.
Loading PPO model from quadrl_var1_ppo.zip ...


C:\Users\Lep\miniconda3\Lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


[Env timing] mujoco_dt=0.010000, frame_skip=5, env_dt=0.050000 sec
Episode  1 | forward = 38.612 | energy = 48.736 | steps = 995
[Env timing] mujoco_dt=0.010000, frame_skip=5, env_dt=0.050000 sec
Episode  2 | forward = 0.284 | energy = 2.186 | steps = 54
[Env timing] mujoco_dt=0.010000, frame_skip=5, env_dt=0.050000 sec
Episode  3 | forward = 35.711 | energy = 48.411 | steps = 995
[Env timing] mujoco_dt=0.010000, frame_skip=5, env_dt=0.050000 sec
Episode  4 | forward = 38.485 | energy = 48.644 | steps = 995
[Env timing] mujoco_dt=0.010000, frame_skip=5, env_dt=0.050000 sec
Episode  5 | forward = 25.945 | energy = 47.835 | steps = 982
[Env timing] mujoco_dt=0.010000, frame_skip=5, env_dt=0.050000 sec
Episode  6 | forward = 0.273 | energy = 2.369 | steps = 57
[Env timing] mujoco_dt=0.010000, frame_skip=5, env_dt=0.050000 sec
Episode  7 | forward = 28.151 | energy = 48.497 | steps = 995
[Env timing] mujoco_dt=0.010000, frame_skip=5, env_dt=0.050000 sec
Episode  8 | forward = 38.399 | ener

In [22]:
### Metrics Cycle Smoothness
import numpy as np
import Diffusion as diff


def summarize_over_rollouts(list_of_q, angle_dims=None, eps=1e-6):
    acc = []
    jerk = []
    acc_mean = []
    acc_norm = []
    acc_norm_mean = []
    jerk_mean = []
    jerk_norm = []
    jerk_norm_mean = []

    for q in list_of_q:
        q = np.asarray(q, dtype=np.float32)

        # differences
        dq = q[1:] - q[:-1]
        if angle_dims is not None:
            dq[:, angle_dims] = (dq[:, angle_dims] + np.pi) % (2*np.pi) - np.pi

        ddq = dq[1:] - dq[:-1]
        if angle_dims is not None:
            ddq[:, angle_dims] = (ddq[:, angle_dims] + np.pi) % (2*np.pi) - np.pi

        dddq = ddq[1:] - ddq[:-1]
        if angle_dims is not None:
            dddq[:, angle_dims] = (dddq[:, angle_dims] + np.pi) % (2*np.pi) - np.pi

        # Per joint RMS
        rms_vel  = np.sqrt(np.mean(dq**2, axis=0)) 
        rms_acc  = np.sqrt(np.mean(ddq**2, axis=0))  
        rms_jerk = np.sqrt(np.mean(dddq**2, axis=0))  

        acc.append(rms_acc)
        jerk.append(rms_jerk)

        acc_mean.append(float(rms_acc.mean()))
        jerk_mean.append(float(rms_jerk.mean()))


        # Amplitude normalized 
        rms_vel_mean = float(rms_vel.mean())
        
        acc_norm_joint = rms_acc / (rms_vel + eps)
        acc_norm.append(acc_norm_joint)
        acc_norm_mean.append(float(acc_norm_joint.mean()))

        jerk_norm_joint = rms_jerk / (rms_vel + eps)
        jerk_norm.append(jerk_norm_joint)
        jerk_norm_mean.append(float(rms_jerk.mean() / (rms_vel_mean + eps)))

    acc = np.stack(acc, axis=0)
    jerk = np.stack(jerk, axis=0)
    jerk_norm = np.stack(jerk_norm, axis=0)
    acc_norm = np.stack(acc_norm, axis=0)

    return {
        "accel_per_joint_mean": acc.mean(axis=0),
        "accel_per_joint_std":  acc.std(axis=0),
        "jerk_per_joint_mean":  jerk.mean(axis=0),
        "jerk_per_joint_std":   jerk.std(axis=0),
        "accel_global_mean": float(np.mean(acc_mean)),
        "accel_global_std":  float(np.std(acc_mean)),
        "jerk_global_mean":  float(np.mean(jerk_mean)),
        "jerk_global_std":   float(np.std(jerk_mean)),
        "accel_norm_per_joint_mean": acc_norm.mean(axis=0),
        "accel_norm_per_joint_std":  acc_norm.std(axis=0),
        "accel_norm_global_mean": float(np.mean(acc_norm_mean)),
        "accel_norm_global_std":  float(np.std(acc_norm_mean)),
        "jerk_norm_per_joint_mean": jerk_norm.mean(axis=0),
        "jerk_norm_per_joint_std":  jerk_norm.std(axis=0),
        "jerk_norm_global_mean": float(np.mean(jerk_norm_mean)),
        "jerk_norm_global_std":  float(np.std(jerk_norm_mean)),
    }

quad_morph = [
    0.141421, 0.282843,
    0.141421, 0.282843,
    0.141421, 0.282843,
    0.141421, 0.282843,
]

imitation_obs_indices = list(range(5, 13))

data = np.load("quadruped_var1p_trajectories.npz")
obs   = data["obs"]  
ep_id = data["ep_id"]
morph = data["morph"]  

morph_dim = morph.shape[1]
base_start = morph_dim + 2  
base_obs = obs[:, base_start:] 

# only the joint pose dims used for imitation/diffusion:
pose = base_obs[:, imitation_obs_indices].astype(np.float32)  # (T, D_target)

print("base_obs shape:", base_obs.shape)
print("pose shape:", pose.shape)

rollouts = []
for eid in np.unique(ep_id):
    ep_pose = pose[ep_id == eid]

    if ep_pose.shape[0] >= 200:
        rollouts.append(ep_pose[:200])

diff_model, meta, device = load_diffusion(
    weights_path="quadruped_morph_diffusion_weights.pt",
    traj_file="quadruped_morph_trajectories.npz",)

cycles = []
for i in range(10):
    seq = diff.sample_full_cycle_from_morph(
        diff_model=diff_model,
        meta=meta,
        morph_vec=quad_morph,
        cycle_steps=200,
        device=device,
    )
    cycles.append(np.asarray(seq, dtype=np.float32))

stats = summarize_over_rollouts(
    rollouts,
    angle_dims=np.arange(rollouts[0].shape[1]) )
print("var 1 ppo")
print("RMS accel:", stats["accel_global_mean"])
print("RMS jerk :", stats["jerk_global_mean"])
print("RMS norm accel:", stats["accel_global_mean"])
print("RMS nom jerk :", stats["jerk_global_mean"])



D = cycles[0].shape[1]
stats = summarize_over_rollouts(
    cycles,
     angle_dims=np.arange(D) )

print("generative diff")
print("RMS accel:", stats["accel_global_mean"])
print("RMS jerk :", stats["jerk_global_mean"])
print("RMS norm accel:", stats["accel_norm_global_mean"])
print("RMS norm jerk :", stats["jerk_norm_global_mean"])
        

base_obs shape: (42968, 87)
pose shape: (42968, 8)
Using device: cuda
Loaded config from checkpoint:
  traj_file : quadruped_morph_trajectories.npz
  past_len  : 10
  future_len: 200
  n_steps   : 100
  hidden    : 256
  imitation_obs_indices: [5, 6, 7, 8, 9, 10, 11, 12]
[QuadrupedDataset] obs_dim=97, morph_dim=8
[QuadrupedDataset] target_dim=8
[QuadrupedDataset] total samples possible=87182
Reconstructed diffusion dims:
  D_target (obs_dim) = 8
  D_cond             = 88
Loaded diffusion model.
var 1 ppo
RMS accel: 1.4629671573638916
RMS jerk : 1.6638646125793457
RMS norm accel: 1.4629671573638916
RMS nom jerk : 1.6638646125793457
generative diff
RMS accel: 0.13566283732652665
RMS jerk : 0.20317601263523102
RMS norm accel: 1.0549116611480713
RMS norm jerk : 1.546083688735962
